In [2]:
with open('keys.txt', 'r') as file:
    # Read the entire contents of the file into a variable
    file_contents = file.read()
    keys = file_contents.split('\n')

import praw

reddit = praw.Reddit(
    client_id=keys[0],
    client_secret=keys[1],
    user_agent=keys[2],
    username=keys[3],
    password=keys[4]
)

In [3]:
subreddit = reddit.subreddit('wallstreetbets')
for post in subreddit.new(limit=10):
    print(post.title)

Nvidia Short or Long ?
This is what a 500% up and 500% down option looks like.
AN: the crew is abandoning their ship
Minnesota Governor to Sign Marijuana Legalization Bill on Tuesday
Jeff Bezos coming in hot with the insider buy on $Amzn. One whole ass share
Just waiting on margin to walk in
With a pelvic thrust, I can tell you reversing Cramer 23 years ago worked and Warren Buffet is one of us.
Had a question about shorting stocks and knowing the outcome before hand.
😂🤣😹
Countdown: 5 Days


In [5]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [6]:
for post in subreddit.new(limit=10):
    doc = nlp(post.title)
    for token in doc:
        print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_)

Nvidia Nvidia PROPN NNP compound
Short Short PROPN NNP ROOT
or or CCONJ CC cc
Long long ADJ JJ conj
? ? PUNCT . punct
This this PRON DT nsubj
is be AUX VBZ ROOT
what what PRON WP det
a a DET DT det
500 500 NUM CD nummod
% % NOUN NN nmod
up up ADV RB advmod
and and CCONJ CC cc
500 500 NUM CD nummod
% % NOUN NN conj
down down NOUN NN prep
option option NOUN NN nsubj
looks look VERB VBZ ccomp
like like ADP IN prep
. . PUNCT . punct
AN an NOUN NN meta
: : PUNCT : punct
the the DET DT det
crew crew NOUN NN nsubj
is be AUX VBZ aux
abandoning abandon VERB VBG ROOT
their their PRON PRP$ poss
ship ship NOUN NN dobj
Minnesota Minnesota PROPN NNP compound
Governor Governor PROPN NNP ROOT
to to PART TO aux
Sign Sign PROPN NNP appos
Marijuana Marijuana PROPN NNP compound
Legalization Legalization PROPN NNP compound
Bill Bill PROPN NNP dobj
on on ADP IN prep
Tuesday Tuesday PROPN NNP pobj
Jeff Jeff PROPN NNP compound
Bezos Bezos PROPN NNP nsubj
coming come VERB VBG acl
in in ADP IN prep
hot hot ADJ 

In [7]:
def extract_stock_information(text):
    doc = nlp(text)
    entities = []
    sentiments = []

    for entity in doc.ents:
        if entity.label_ == "ORG" and "tesla" in entity.text.lower():
            entities.append(entity.text)

    for sentence in doc.sents:
        sentiment = sentence.sentiment
        sentiments.append(sentiment)

    return entities, sentiments

In [8]:
for post in subreddit.new(limit=10):
    entities, sentiments = extract_stock_information(post.title)
    if entities:
        print("Entities: ", entities)
        print("Sentiments: ", sentiments)


In [10]:
import yfinance as yf

def retrieve_historical_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    return stock_data

In [82]:
import datetime as datetime

symbol = "TSLA"
start_date = datetime.datetime(2023, 5, 1)
end_date = datetime.datetime(2023, 6, 1)

stock_data = retrieve_historical_stock_data(symbol, start_date, end_date)
type(stock_data.index[0])

[*********************100%***********************]  1 of 1 completed


pandas._libs.tslibs.timestamps.Timestamp

In [110]:
posts = []
for post in subreddit.new(limit = 100):
    # if start_date.timestamp() <= post.created_utc <= end_date.timestamp():
    posts.append({
        'title': post.title,
        'body': post.selftext,
        'date': post.created_utc,
        'score': post.score,
    })
    # elif post.created_utc < start_date.timestamp():
    #     break
import pandas as pd
from datetime import datetime

posts_df = pd.DataFrame(posts)
posts_df['Date'] = pd.to_datetime(posts_df['date'], unit='s').dt.strftime("%Y-%m-%d")
#posts_df = posts_df.drop('date', axis=1)
posts_df['Date'] = pd.to_datetime(posts_df['Date'])
posts_df.head()


,title,body,date,score,Date
0,"BEWARE when trading options on EBAY, NVDA or a...",,1.685229e+09,11,2023-05-27
1,NVDA just announced a $10b stock offering. Goi...,This is classic. If there’s anything I know ab...,1.685229e+09,4,2023-05-27
2,Biggg Nasdaq guy over here,,1.685228e+09,0,2023-05-27
3,Everyone has a plan.,,1.685228e+09,33,2023-05-27
4,State Farm to stop accepting homeowners insura...,https://www.cnbc.com/2023/05/27/state-farm-to-...,1.685227e+09,0,2023-05-27


In [111]:
def merge_data(posts, stock_data):
    merged_data = pd.merge(posts, stock_data, how='inner', left_on='Date', right_index=True)
    return merged_data

In [112]:
merged_data = merge_data(posts_df, stock_data)
print(merged_data.dtypes)
merged_data.head()

title                object
body                 object
date                float64
score                 int64
Date         datetime64[ns]
Open                float64
High                float64
Low                 float64
Close               float64
Adj Close           float64
Volume                int64
dtype: object


,title,body,date,score,Date,Open,High,Low,Close,Adj Close,Volume
53,Wow…. Of all the times I thought I was going t...,,1.685145e+09,56,2023-05-26,184.619995,198.600006,184.529999,193.169998,193.169998,161765900
54,Elon Musk’s Neuralink says it has received FDA...,,1.685145e+09,15,2023-05-26,184.619995,198.600006,184.529999,193.169998,193.169998,161765900
55,Cope or Accountability?,,1.685145e+09,32,2023-05-26,184.619995,198.600006,184.529999,193.169998,193.169998,161765900
56,The next Nvidia? Barclays analyst reveals the ...,,1.685144e+09,0,2023-05-26,184.619995,198.600006,184.529999,193.169998,193.169998,161765900
57,NVDA short seller dinner looking delicious!,,1.685144e+09,1409,2023-05-26,184.619995,198.600006,184.529999,193.169998,193.169998,161765900


In [113]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Assume merged_data contains the merged DataFrame with Reddit posts and stock price data
# Ensure the DataFrame has columns for 'date', 'title', 'body', 'score', 'Open', 'Close', etc.

# Prepare the features and target variables

features = merged_data[['score', 'Open', 'Volume', 'date']]  # Adjust the features based on your requirements
target = merged_data['Close']  # Adjust the target variable based on your requirements

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

print(X_train.head())
print(y_train.head())

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

# Print the model performance
print("Train Score:", train_score)
print("Test Score:", test_score)

    score        Open     Volume          date
61     23  184.619995  161765900  1.685140e+09
56      0  184.619995  161765900  1.685144e+09
59      0  184.619995  161765900  1.685141e+09
93   1702  184.619995  161765900  1.685114e+09
86    129  184.619995  161765900  1.685119e+09
61    193.169998
56    193.169998
59    193.169998
93    193.169998
86    193.169998
Name: Close, dtype: float64
Train Score: 1.0
Test Score: 1.0
